In [1]:
import numpy as np
import healpy as hp
import glob
import matplotlib.pyplot as plt

from enterprise.pulsar import Pulsar
import skyplot_tools as spt


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
plt.rcParams.update(plt.rcParamsDefault)
params = {'backend': 'pdf',
        'axes.labelsize': 10,
        'lines.markersize': 4,
        'font.size': 10,
        'xtick.major.size':6,
        'xtick.minor.size':3,
        'ytick.major.size':6,
        'ytick.minor.size':3,
        'xtick.major.width':0.5,
        'ytick.major.width':0.5,
        'xtick.minor.width':0.5,
        'ytick.minor.width':0.5,
        'lines.markeredgewidth':1,
        'axes.linewidth':1.2,
        'legend.fontsize': 7,
        'xtick.labelsize': 10,
        'ytick.labelsize': 10,
        'savefig.dpi':200,
        'path.simplify':True,
        #'font.family': 'serif',
        #'font.serif':'Times',
        'text.latex.preamble': [r'\usepackage{amsmath}'],
        'text.usetex':True}

plt.rcParams.update(params)

In [4]:
#The pulsars we'll be analyzing
psrlist = ['J1713+0747',
           'J1909-3744',
           'J1640+2224',
           'J1600-3053',
           'J2317+1439',
           'J1918-0642',
           'J1614-2230',
           'J1744-1134',
           'J0030+0451',
           'J2145-0750',
           'J1857+0943',
           'J1853+1303',
           'J0613-0200',
           'J1455-3330',
           'J1741+1351',
           'J2010-1323',
           'J1024-0719',
           'J1012+5307',
           'J0437-4715',
           'J0034-0534',
          ]

In [ ]:
datadir = '/Users/ptb/Projects/nanograv/data/nano11'
ephem = 'DE436'

with open(datadir + '/11y_psrs.txt', 'r') as f:
    psrlist = [line.strip() for line in f]

# get PSR locations

ONLY NEED TO DO ONCE PER psrlist

In [91]:
parfiles = sorted(glob.glob(datadir + '/*.par'))
timfiles = sorted(glob.glob(datadir + '/*.tim'))

# filter
parfiles = [x for x in parfiles if x.split('/')[-1].split('_')[0] in psrlist]
timfiles = [x for x in timfiles if x.split('/')[-1].split('_')[0] in psrlist]

psrs = []
for p, t in zip(parfiles, timfiles):
    psr = Pulsar(p, t, ephem=ephem)
    psrs.append(psr)

In [92]:
with open('11yr_locs.txt', 'w') as f:
    for p in psrs:
        phi = p.phi
        theta = np.pi/2 - p.theta
        f.write('{:s} {:f} {:f}\n'.format(p.name, phi, theta))

# Plot

In [99]:
psr_dict

{'B1855+09': [4.95337, 0.168487],
 'B1937+21': [5.137725, 0.37467],
 'B1953+29': [5.207391, 0.506356],
 'J0023+0923': [0.101584, 0.163886],
 'J0030+0451': [0.132894, 0.084841],
 'J0340+4130': [0.961625, 0.724531],
 'J0613-0200': [1.630718, -0.035135],
 'J0645+5158': [1.771442, 0.907062],
 'J1012+5307': [2.672786, 0.927072],
 'J1024-0719': [2.725526, -0.127795],
 'J1455-3330': [3.908663, -0.58491],
 'J1600-3053': [4.192565, -0.539255],
 'J1614-2230': [4.252532, -0.392851],
 'J1640+2224': [4.364541, 0.390996],
 'J1643-1224': [4.379188, -0.216705],
 'J1713+0747': [4.510915, 0.136026],
 'J1738+0333': [4.620321, 0.062012],
 'J1741+1351': [4.631751, 0.241942],
 'J1744-1134': [4.644714, -0.202142],
 'J1747-4036': [4.659209, -0.708869],
 'J1853+1303': [4.947813, 0.227979],
 'J1903+0327': [4.9877, 0.060307],
 'J1909-3744': [5.016908, -0.658642],
 'J1910+1256': [5.018527, 0.225853],
 'J1918-0642': [5.056221, -0.117106],
 'J1923+2515': [5.07618, 0.440892],
 'J1944+0907': [5.166853, 0.159227],
 'J

In [7]:
psr_dict = {}
with open('11yr_locs.txt', 'r') as f:
    for line in f:
        line = line.split()
        psr_dict[line[0]] = [float(line[1]), float(line[2])]
    
psr_locs = []
for p in psrlist:
    psr_locs.append(psr_dict[p])
psr_locs = np.array(psr_locs)

In [8]:
nside = 8 #corresponds to 768 pixels
npix = hp.nside2npix(nside)

In [11]:
skymap = np.loadtxt('ULs.dat', usecols=(1))
skymap_smooth = hp.smoothing(skymap, 0.75)

Sigma is 1094.907479 arcmin (0.318496 rad) 
-> fwhm is 2578.310078 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin


In [14]:
plt.figure(0)
ax = plt.subplot(111, projection='astro mollweide')
ax.grid()
spt.outline_text(ax)

spt.healpix_heatmap(skymap)

ax.plot(psr_locs[:,0], psr_locs[:,1], '*', color='w', markersize=10, mew=1.0, mec='k', alpha=0.9)    
ax.set_title(r'11yr BWM ULs')

plt.colorbar(orientation='horizontal')
plt.suptitle(r'95\% Characteristic Strain Upper Limit, $A_{95}$', y=0.0)
plt.grid(linestyle='dotted', color='k')
plt.tight_layout()
plt.savefig('skymap.png')


/Users/ptb/anaconda/envs/enterprise/lib/python3.6/site-packages/matplotlib/projections/geo.py:485: RuntimeWarning: invalid value encountered in arcsin
  theta = np.arcsin(y / np.sqrt(2))


In [16]:
plt.clf()
plt.figure(1)
ax = plt.subplot(111, projection='astro mollweide')
ax.grid()
spt.outline_text(ax)

spt.healpix_heatmap(skymap_smooth)

ax.plot(psr_locs[:,0], psr_locs[:,1], '*', color='w', markersize=10, mew=1.0, mec='k', alpha=0.9)    
ax.set_title(r'11yr BWM UL')

plt.colorbar(orientation='horizontal')
plt.suptitle(r'95\% Characteristic Strain Upper Limit, $A_{95}$', y=0.0)
plt.grid(linestyle='dotted', color='k')
plt.tight_layout()
plt.savefig('skymap_smooth.png')

/Users/ptb/anaconda/envs/enterprise/lib/python3.6/site-packages/matplotlib/projections/geo.py:485: RuntimeWarning: invalid value encountered in arcsin
  theta = np.arcsin(y / np.sqrt(2))
